### Have to derep the MAGs
- At strain level (99%)
- At Species level (95%)

Total number of current MAGs (810 SRAs) 10,544

For drep quality file:
- https://drep.readthedocs.io/en/master/advanced_use.html#using-external-genome-quality-information
- https://github.com/MrOlm/drep/issues/220
- https://genomicsaotearoa.github.io/metagenomics_summer_school/resources/1_APPENDIX_ex8_Dereplication/#step-3



In [ ]:
# MAG location:
/group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/MAGs/genomes/all_genomes/*.fasta
# quality report location:
/group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/MAGs/250410_mag_quality.tsv

### Quality file for dRep:
awk -F'\t' 'BEGIN {OFS=","} {print $1, $2, $4}' /group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/MAGs/250410_mag_quality.tsv > 250410_mag_qual.drep.csv

# In the new file convert headings to: genome,completeness,contamination
# drep also needs the trailing .fasta 
echo "genome,completeness,contamination" > 250411_dRep.genomeInfo
cut -f1,2,4 /group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/MAGs/250411_mag_quality.tsv | sed 's/\t/.fasta\t/' | sed 's/\t/,/g' | tail -n+2 >> 250411_dRep.genomeInfo

In [ ]:
# use drep
srun --account=ctbrowngrp -p high2 -J drep -t 15:00:00 -c 32 --mem=80gb --pty bash

mamba activate drep
dRep dereplicate \
drep.95 \
--genomeInfo 250411_dRep.genomeInfo \
-p 32 \
-g /group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/MAGs/genomes/all_genomes/*.fasta \
-pa 0.9 -sa 0.95 -nc 0.30 -cm larger \
-comp 50 -con 10 -l 1000 


In [ ]:
# make sourmash zip and index as rocksdb
srun --account=ctbrowngrp -p high2 -J indexdb -t 5:00:00 -c 1 --mem=50gb --pty bash

sourmash index MAGs.99.k21.rocksdb -F rocksdb -k 21 *.zip

### Dereplicating viruses and plasmids:
- 1,249,984 viral sequences --> 446,191 clusters
- 425,836 plasmids --> 168,278 clusters


In [ ]:
# first remove all contigs with a viral score < 0.9
# then concat
cat /group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/genomad/viral/*rename.fa > 250418_vi
ral_sequences.genomad.fa

cat /group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/genomad/plasmid/*rename.fa > 250418_plasmid_sequences.genomad.fa
# then drep

In [ ]:
# drep doesnt work, too many contigs. Use cdhit
# and sort it by length 
mamba activate bbmap
sortbyname.sh in=250418_plasmid_sequences.genomad.fa \
out=250418_plasmid_sequences.genomad.sort.fa length=descending

# run cdhit
mamba activate cdhit
cd-hit-est -i 250418_plasmid_sequences.genomad.sort.fa \
-o 250419_plasmid_sequences.95.cluster.fa -d 0 \
-c 0.95 -aS 0.85 -M 95000 -T 48

In [ ]:
mamba activate bbmap
sortbyname.sh in=250418_viral_sequences.genomad.fa \
out=250418_viral_sequences.genomad.sort.fa length=descending

# run cdhit
mamba activate cdhit
cd-hit-est -i 250418_viral_sequences.genomad.sort.fa \
-o 250419_viral_sequences.95.cluster.fa -d 0 \
-c 0.95 -aS 0.85 -M 98000 -T 48

In [ ]:
# drep code (too many seqs for drep)
# split contigs into individual fastas (dRep accepts individual contigs only)
mkdir contigs
cd ./contigs
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' ../250418_viral_sequences.genomad.fa 

# can also do with seqkit
srun --account=ctbrowngrp -p bmh -J drep_vir -t 1:00:00 -c 4 --mem=40gb --pty bash

mamba activate seqkit
seqkit split -i -f your_multifasta.fasta


srun --account=ctbrowngrp -p bmh -J drep_vir -t 40:00:00 -c 48 --mem=100gb --pty bash
srun --account=ctbrowngrp -p high2 -J drep_plasmid -t 24:00:00 -c 48 --mem=100gb --pty bash


mamba activate dRep
# Run dRep at 95% ANI over 85% of length of longest contigs
dRep dereplicate ./drep --S_algorithm ANImf \
--ignoreGenomeQuality -l 10000 -sa 0.95 -nc 0.85 \
-g ./250418_viral_sequences.genomad.fa.split/*.fa -p 48


dRep dereplicate ./drep --S_algorithm ANImf \
--ignoreGenomeQuality -l 500 -sa 0.95 -nc 0.85 \
-g ./250418_plasmid_sequences.genomad.fa.split/*.fa -p 48

In [ ]:
# cdhit the contigs that are MAGS AND plasmids
# run cdhit
mamba activate cdhit
cd-hit-est -i 250418_viral_sequences.genomad.sort.fa \
-o 250419_viral_sequences.95.cluster.fa -d 0 \
-c 0.95 -aS 0.85 -M 98000 -T 48



In [ ]:

srun --account=ctbrowngrp -p bmh -J drep -t 4:00:00 -c 32 --mem=100gb --pty bash

cd-hit-est-2d -i ../../drep/drep.95/all_contigs.fa \
-i2 250419_plasmid_sequences.95.cluster.fa \
-o 250422_novel_plasmid.fa -d 0 \
-c 1 -aS 0.95 -M 79000 -T 32